In [64]:
import spacy
import os
import sklearn
import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS
import readability
import json
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import textacy
from sklearn.metrics import confusion_matrix
import gensim
from numpy import dot
from numpy.linalg import norm
import neuralcoref
nlp = spacy.load("en")
nlp.max_length = 15000000
neuralcoref.add_to_pipe(nlp)

In [65]:
with open("./summary_quality/train_data.json",'r') as fin:
    train_content = json.load(fin)

In [66]:
with open("./summary_quality/test_data.json",'r') as fin:
    test_content = json.load(fin)

In [68]:
def getFeature1(doc):
    nounChunks = {}
    for chunk in doc.noun_chunks:
        c = nounChunks.get(chunk,0)
        nounChunks[chunk] = c+1
        if c+1 > 1:
            print(nounChunks)
    return len([val for val in nounChunks.values() if val > 1])

In [69]:
def getFeature2(doc):
    if doc._.has_coref:
        return len(doc._.coref_clusters)
    else :
        return 0

In [70]:
docFeature = OrderedDict()
for file in os.listdir("./summary_quality/summaries/"):
#     print(file)
    f = open("./summary_quality/summaries/"+file, encoding = "ISO-8859-1")
    text = f.read().lower()
    doc = nlp(text)
    wordList=[]
    for token in doc:
        if token.pos_ != "PUNCT" and token.pos_ != "SPACE" and token.pos_ != "SYM":
            wordList.append(token.text.lower())
    doc = nlp(" ".join(wordList))
    t = (getFeature1(doc),getFeature2(doc))
    docFeature[file] = t
    f.close()

In [81]:
#create train and test data
feature_X_Y = OrderedDict()
for k,v in sorted(train_content.items(), key=lambda x: x):
    feature_X_Y[k] = (docFeature[k],v['coherence'])
test_X_Y = OrderedDict()
for k,v in sorted(test_content.items(), key=lambda x: x):
    test_X_Y[k] = (docFeature[k],v['coherence'])
test_x = []
test_y = []
for k,v in test_X_Y.items():
    test_x.append([float(v[0][0]),float(v[0][1])])
    test_y.append(float(v[1]))
    train_x = []
train_y = []
for k,v in feature_X_Y.items():
    train_x.append([float(v[0][0]),float(v[0][1])])
    train_y.append(float(v[1]))

In [82]:
regressor = SVR(kernel='rbf')
regressor.fit(train_x,train_y)
y_pred = regressor.predict(test_x)
print("SVR MSE ",mean_squared_error(test_y,y_pred))
print("SVR Pearson ",pearsonr(test_y,y_pred))

SVR MSE  1.0543990943762727
SVR Pearson  (-0.08277872745551723, 0.2524189297053374)


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
